## The Script
For the evaluation of the segmentation metrics is highly based on the helper functions provided by Janesick et al. so we placed it in `Xenium Segmentation Transfer/segmentation-comparison.ipynb` 
Some remarks on the evaluation can be found below:


## ENACT vs Xenium
Are the two segmentations e compare. Bin2cell as well as Spaceranger v4.0 both rely on stardist nucleus segmentation and subsequent circular expansion so I assume they would be similar enough. This neglects the following violations: 
- This assumption does not hold true for bin2cell as it also relies on the Total Gene Expression image for segmentation.
- Also for Spaceranger it is not fully valid as 10X claims to have fine-tuned their stardist model so the nucleus segmentation could be slightly varying wrt. the public models.
